## pytorch

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Assuming the ResNet class is already defined
# model = ResNet()

# Load the model
model = ResNet()  # Create an instance of the model class
model.load_state_dict(torch.load('garbage_classification_model_cpu.pth', map_location='cpu'))

# Put the model in evaluation mode
model.eval()

def predict(image_path):
    # Define the same transformations applied on the training data
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # same as the ones during training & converting to tensor
        transforms.ToTensor(),
    ])

    # Open the image and apply the transformations
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Forward pass
    with torch.no_grad():  # Deactivate gradients for prediction
        outputs = model(image)
        # If the model outputs logits, apply softmax to get probabilities
        predictions = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(predictions, dim=1)

    return predicted_class.item()

# Use the function


In [ ]:
image_path = "path_to_your_image.jpg"
print(f"Predicted class: {predict(image_path)}")

## ONNX

In [9]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

def predict_onnx(image_path):
    # Start an inference session
    ort_session = ort.InferenceSession('C:\\Users\\wilpg\\Documents\\GitHub\\HackHarvard2023\\model\\trained_models\\garbage_classification_model.onnx')

    # Define the same transformations applied during training
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    # Open the image and apply the transformations
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).numpy()  # Convert to numpy array

    # Prepare input data (matching the input name from when we exported the model)
    ort_inputs = {ort_session.get_inputs()[0].name: image}

    # Forward pass
    ort_outs = ort_session.run(None, ort_inputs)
    predictions = ort_outs[0]

    # If the model outputs logits, you might want to apply a softmax function to get probabilities
    # However, if it's a multi-class classification, you can directly get the final predicted class like this
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class[0]

# Use the function

In [16]:
image_path = "C:\\Users\\wilpg\\Documents\\GitHub\\HackHarvard2023\\data\\garbage_classification\\trash\\trash1.jpg"
print(f"Predicted class: {predict_onnx(image_path)}")


Predicted class: 5
